In [8]:
import numpy as np
from functools import reduce 

def initialize_parameters(layer_dims):
    parameters = {}
    for l in range(1, len(layer_dims)):
        parameters[f"W{l}"] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters[f"b{l}"] = np.zeros((layer_dims[l], 1))
    return parameters

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    s = sigmoid(x)
    return s * (1 - s)

def modified_Y(Y):
    modf_Y = np.zeros((Y.size, Y.max() + 1))
    modf_Y[np.arange(Y.size), Y] = 1
    modf_Y = modf_Y.T
    return modf_Y

def forward_propagation(X, parameters):
    caches = []
    A = X
    for l in range(1, len(parameters) // 2):
        W, b = parameters[f"W{l}"], parameters[f"b{l}"]
        Z = np.dot(W, A) + b
        A = sigmoid(Z)
        caches.append((A, W, b, Z))
    
    WL, bL = parameters[f"W{len(parameters) // 2}"], parameters[f"b{len(parameters) // 2}"]
    AL = sigmoid(np.dot(WL, A) + bL)
    caches.append((AL, WL, bL, np.dot(WL, A) + bL))
    return AL, caches

def backward_propagation(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))

    A_prev, W, b, Z = caches[L - 1]
    dZ = dAL * sigmoid_prime(Z)
    grads[f"dW{L}"] = np.dot(dZ, A_prev.T) / m
    grads[f"db{L}"] = np.sum(dZ, axis=1, keepdims=True) / m
    grads[f"dA{L - 1}"] = np.dot(W.T, dZ)

    for l in reversed(range(1, L - 1)):
        A_prev, W, b, Z = caches[l]
        dZ = grads[f"dA{L - 1}"] * sigmoid_prime(Z)  # Fix indexing here
        grads[f"dW{l}"] = np.dot(dZ, A_prev.T) / m
        grads[f"db{l}"] = np.sum(dZ, axis=1, keepdims=True) / m
        grads[f"dA{l - 1}"] = np.dot(W.T, dZ)

    # Handle the first layer separately
    A0, W0, b0, Z0 = caches[0]
    dZ0 = grads["dA0"] * sigmoid_prime(Z0)
    grads["dW0"] = np.dot(dZ0, A0.T) / m
    grads["db0"] = np.sum(dZ0, axis=1, keepdims=True) / m

    return grads

def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = - (1 / m) * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
    return cost

def update_parameters(parameters, grads, learning_rate):
    for l in range(1, len(parameters) // 2 + 1):
        parameters[f"W{l}"] -= learning_rate * grads[f"dW{l}"]
        parameters[f"b{l}"] -= learning_rate * grads[f"db{l}"]

    return parameters

def train_iteration(i, parameters, X, Y, learning_rate):
    AL, caches = forward_propagation(X, parameters)
    cost = compute_cost(AL, Y)
    grads = backward_propagation(AL, Y, caches)
    parameters = update_parameters(parameters, grads, learning_rate)

    if i % 100 == 0:
        print(f'Cost after iteration {i}: {cost}')

    return parameters

modf_Y = modified_Y(Y)

def train_neural_network(X, Y, layer_dims, learning_rate, num_iterations):
    parameters = initialize_parameters(layer_dims)
    trained_parameters = reduce(lambda params, i: train_iteration(i, params, X, Y, learning_rate),
                                range(num_iterations),
                                parameters)
    return trained_parameters

# Load and preprocess data (simplified for demonstration)
import numpy as np
import pandas as pd

# Load and preprocess data
data = pd.read_csv("./MNIST.csv")
data = np.array(data)
np.random.shuffle(data)

m, n = data.shape

Y = data[:, 0]
X = data[:, 1:]
X = X / 255.0

num_test_samples = 1000
X_test = X[:num_test_samples, :].T
Y_test = Y[:num_test_samples]

X_train = X[num_test_samples:, :].T
Y_train = Y[num_test_samples:]
Y_train = Y_train.reshape(1, -1)
X_train = X_train.T
X_test = X_test.T

# Define neural network architecture
layer_dims = [X_train.shape[0], 20, 10, 1]




# Training function
def train_neural_network(X, modf_Y, layer_dims, learning_rate, num_iterations):
    parameters = initialize_parameters(layer_dims)
    for i in range(num_iterations):
        AL, caches = forward_propagation(X, parameters)
        cost = compute_cost(AL, Y)
        grads = backward_propagation(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)

        if i % 100 == 0:
            print(f'Cost after iteration {i}: {cost}')

    return parameters

# Train the neural network
trained_parameters = train_neural_network(X_train, Y_train, layer_dims, learning_rate=0.01, num_iterations=1000)

# Test the neural network on the test set
test_predictions, _ = forward_propagation(X_test, trained_parameters)
test_predictions = (test_predictions > 0.5).astype(int)

# Evaluate the accuracy
accuracy = np.mean(test_predictions == Y_test)
print(f"Accuracy on test set: {accuracy}")


IndexError: tuple index out of range